# Start working on LLM explanation

In [28]:
import dill
import shap
import pandas as pd
import numpy as np
import os

In [12]:
model_path = "../models/model_winner.dill"
preprocessor_path = "../models/preprocessor.dill"
data_path = "../raw_data/data_cleaned.csv"

model_path, preprocessor_path, data_path

('../models/model_winner.dill',
 '../models/preprocessor.dill',
 '../raw_data/data_cleaned.csv')

In [14]:
# load the data
df = pd.read_csv(data_path)
df.head(1)

,title,company_profile,description,requirements,benefits,has_company_logo,employment_type,industry,function,fraudulent,country
0,market intern,food weve create groundbreaking awardwinning c...,food fastgrowing jam beard awardwinning online...,experience content management systems major pl...,miss value,1,Other,NaN,Marketing,0,US


In [7]:
# load the model and preprocessor
with open(model_path, "rb") as f:
    model = dill.load(f)

with open(preprocessor_path, "rb") as f:
    preprocessor = dill.load(f)

print("Model:", model)
print("Preprocessor loaded:", type(preprocessor))

Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=11, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=275, n_jobs=2,
              num_parallel_tree=None, ...)
Preprocessor loaded: <class 'sklearn.compose._column_transformer.ColumnTransformer'>


# Streamlit Output stimulation

In [23]:
sample = df.sample(1, random_state=42)
sample

,title,company_profile,description,requirements,benefits,has_company_logo,employment_type,industry,function,fraudulent,country
4708,python engineer,miss value,stylect dynamic startup help help women discov...,’ care study gpa college want someone isstrong...,negotiable salary potential equity right candi...,1,Full-time,Apparel & Fashion,Information Technology,0,GB


In [24]:
# remove target if present
X_sample = sample.drop(columns=["fraudulent"], errors="ignore")

# sanity check
X_sample

,title,company_profile,description,requirements,benefits,has_company_logo,employment_type,industry,function,country
4708,python engineer,miss value,stylect dynamic startup help help women discov...,’ care study gpa college want someone isstrong...,negotiable salary potential equity right candi...,1,Full-time,Apparel & Fashion,Information Technology,GB


In [25]:
X_transformed = preprocessor.transform(X_sample)

X_transformed.shape

(1, 5265)

In [ ]:
prediction = model.predict(X_transformed)[0]
probability = model.predict_proba(X_transformed)[0][1]

label = "FAKE" if prediction == 1 else "REAL"

print(f"Prediction: {label}")
print(f"Fake probability: {probability:.3f}")


Prediction: REAL
Fake probability: 0.000


# SHAP 

In [37]:
type(model)

xgboost.sklearn.XGBClassifier

In [38]:
model

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [36]:
explainer = shap.TreeExplainer(
    model,
    model_output="raw"
)

shap_values = explainer(X_transformed)

ValueError: could not convert string to float: '[4.8447985E-2]'